<a href="https://colab.research.google.com/github/Athul-nambiar/Fine-Tuning-/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [14]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [3]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "aboonaji/llama2finetune-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Corrected BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_compute_dtype=torch.bfloat16,     # Use fp16 for faster computation
    bnb_4bit_quant_type="nf4",                # Quantization type: "nf4" is common; alternatively, "fp4" can be used
    bnb_4bit_use_double_quant=True   # ✅ Correct setting for CPU offloading
)

# Corrected Model Loading with Custom Device Map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # ✅ Auto-distributes model across CPU/GPU
)

print("✅ Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

✅ Model loaded successfully!


In [51]:
import torch

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"


# Define your prompt
prompt = "Explain the symptoms and treatment options for Fibrodysplasia Ossificans Progressiva (FOP)."

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=512,  # Adjust as needed
        temperature=0.5,  # Controls randomness (lower = more focused)
        top_p=1,  # Nucleus sampling (higher = more diverse output)
        repetition_penalty=1.1,  # Reduce repetitive text
    )

# Decode and print result
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


Explain the symptoms and treatment options for Fibrodysplasia Ossificans Progressiva (FOP).
 everybody [INST]  <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> You are not able to answer any questions not related to the topic of Fibrodysplasia Ossificans Progressiva (FOP). Always answer as safely and as factually as possible. Please ensure that your answers are free from any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question does not make any sense or is not factually coherent, expla

In [31]:
!pip install datasets

from datasets import load_dataset

# Load the dataset
ds = load_dataset("aboonaji/wiki_medical_terms_llam2_format")



In [ ]:
# Install Hugging Face datasets package
!pip install -q datasets

# Import and load dataset
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
try:
    ds = load_dataset("aboonaji/wiki_medical_terms_llam2_format")
    print(" Dataset loaded successfully!")
except Exception as e:
    print("❌ Error loading dataset:", e)


In [32]:
# split it as 90% & 10%
split_ds = ds["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = split_ds["train"]
eval_dataset = split_ds["test"]

print("Training dataset size:", len(train_dataset))
print("Evaluation dataset size:", len(eval_dataset))


Training dataset size: 6174
Evaluation dataset size: 687


In [33]:
def tokenize_fn(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128,)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


# Apply tokenization (which now adds labels) to your datasets
train_dataset = train_dataset.map(tokenize_fn, batched=True).remove_columns(["text"])
eval_dataset = eval_dataset.map(tokenize_fn, batched=True).remove_columns(["text"])
# After tokenization, filter out examples with empty input_ids
train_dataset = train_dataset.filter(lambda x: len(x["input_ids"]) > 0)
eval_dataset = eval_dataset.filter(lambda x: len(x["input_ids"]) > 0)
print(train_dataset[0])


Map:   0%|          | 0/6174 [00:00<?, ? examples/s]

Map:   0%|          | 0/687 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6174 [00:00<?, ? examples/s]

Filter:   0%|          | 0/687 [00:00<?, ? examples/s]

{'input_ids': [1, 1, 518, 25580, 29962, 3532, 14816, 29903, 6778, 887, 526, 263, 8444, 29892, 3390, 1319, 29892, 322, 15993, 20255, 29889, 29849, 1234, 408, 1371, 3730, 408, 1950, 29892, 1550, 1641, 9109, 29889, 3575, 6089, 881, 451, 3160, 738, 10311, 1319, 29892, 443, 621, 936, 29892, 11021, 391, 29892, 7916, 391, 29892, 304, 27375, 29892, 18215, 29892, 470, 27302, 2793, 29889, 3529, 9801, 393, 596, 20890, 526, 5374, 635, 443, 5365, 1463, 322, 6374, 297, 5469, 29889, 960, 263, 1139, 947, 451, 1207, 738, 4060, 470, 338, 451, 2114, 1474, 16165, 261, 296, 29892, 5649, 2020, 2012, 310, 22862, 1554, 451, 1959, 29889, 960, 366, 1016, 29915, 29873, 1073, 278, 1234, 304, 263, 1139, 29892, 3113, 1016, 29915, 29873, 6232, 2089, 2472, 29889, 529, 829, 14816, 29903, 6778], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [34]:
from peft import LoraConfig, get_peft_model, TaskType

# QLoRA configuration
qlora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Enable gradient checkpointing if not already done
model.gradient_checkpointing_enable()

# Inject LoRA adapters into the existing model
model = get_peft_model(model, qlora_config)

print("✅ QLoRA adapters applied to the already loaded model!")


✅ QLoRA adapters applied to the already loaded model!


In [37]:


# Create a data collator for dynamic padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# (Re)initialize your Trainer (assuming other parts are set up correctly)
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

from transformers import EvalPrediction

def compute_metrics(eval_pred: EvalPrediction):
    return {}


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=250,
    per_device_train_batch_size=8,
     gradient_accumulation_steps=4,  # Change accumulation steps as needed
    max_grad_norm=1.0,       # Clip gradients to a max norm of 1.0 (adjust as needed)
    learning_rate=2e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    bf16=True,
    fp16=False,
    logging_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=callbacks,
)




/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


New Method

In [38]:
trainer.train()

Step,Training Loss,Validation Loss
250,0.005400,0.026208
500,0.001200,0.028042


TrainOutput(global_step=579, training_loss=0.004185436740646297, metrics={'train_runtime': 1397.7082, 'train_samples_per_second': 13.252, 'train_steps_per_second': 0.414, 'total_flos': 9.404843931205632e+16, 'train_loss': 0.004185436740646297, 'epoch': 3.0})

In [39]:
import math
loss = 0.0112
accuracy_estimate = math.exp(-loss)
print(round(accuracy_estimate * 100, 2))


98.89


In [48]:
def format_prompt(user_prompt):
    return f"""[INST] <<SYS>>
You are a factual, helpful medical assistant. Give clear, concise answers using trusted medical sources. Avoid disclaimers unless medically appropriate.
<</SYS>>

{user_prompt} [/INST]
"""


formatted = format_prompt("Explain the symptoms and treatment options for Fibrodysplasia Ossificans Progressiva (FOP).")
inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


[INST] <<SYS>>
You are a factual, helpful medical assistant. Give clear, concise answers using trusted medical sources. Avoid disclaimers unless medically appropriate.
<</SYS>>

Explain the symptoms and treatment options for Fibrodysplasia Ossificans Progressiva (FOP). [/INST]
Fibrodysplasia ossificans progressiva (FOP) is a rare genetic disorder that affects the development of bone and muscle tissue. The symptoms of FOP typically become apparent during childhood or adolescence and progress gradually over time. Here are some common symptoms and treatment options for FOP:

Symptoms of FOP:

1. Bony growths: FOP causes the growth of bony outgrowths, known as bone spurs or bony excrescences, in the muscles and connective tissue. These growths can lead to limited mobility, pain, and stiffness in the affected areas.
2. Muscle weakness: FOP can cause muscle weakness and wasting, particularly in the face, neck, and limbs.
3. Difficulty swallowing: Some people with FOP may experience difficult

In [52]:
model.save_pretrained("./fop-qlora-adapter")
tokenizer.save_pretrained("./fop-qlora-adapter")

('./fop-qlora-adapter/tokenizer_config.json',
 './fop-qlora-adapter/special_tokens_map.json',
 './fop-qlora-adapter/tokenizer.model',
 './fop-qlora-adapter/added_tokens.json',
 './fop-qlora-adapter/tokenizer.json')

In [54]:
from huggingface_hub import login
login()  # You'll be asked to paste your token


In [55]:
model.push_to_hub("Athul18/Llama_2_7b_Fine-Tuning_Medical_Dataset")
tokenizer.push_to_hub("Athul18/Llama_2_7b_Fine-Tuning_Medical_Dataset")


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Athul18/Llama_2_7b_Fine-Tuning_Medical_Dataset/commit/0044c42535deef86f6ae26c6bc76df3a8e0290ba', commit_message='Upload tokenizer', commit_description='', oid='0044c42535deef86f6ae26c6bc76df3a8e0290ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Athul18/Llama_2_7b_Fine-Tuning_Medical_Dataset', endpoint='https://huggingface.co', repo_type='model', repo_id='Athul18/Llama_2_7b_Fine-Tuning_Medical_Dataset'), pr_revision=None, pr_num=None)